In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [3]:
date = "2017.03.01"
url_part1 = """http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
url_part2 = """&sort=&series=&mdayCnt=0&currentPage="""

url = url_part1+ date + url_part2




df = pd.DataFrame()
df_temp = pd.DataFrame()
df_temp = df_temp.append({"Cat":"","ComName":"","Content":"","coID":"","pubDate":"","pubTime":"","rcpNo":"","reqDate":""},ignore_index=True)

cat_idx = df_temp.columns.get_loc("Cat")
conm_idx = df_temp.columns.get_loc("ComName")
con_idx = df_temp.columns.get_loc("Content")
id_idx = df_temp.columns.get_loc("coID")
pdate_idx = df_temp.columns.get_loc("pubDate")
ptime_idx = df_temp.columns.get_loc("pubTime")
rcp_idx = df_temp.columns.get_loc("rcpNo")
rdate_idx = df_temp.columns.get_loc("reqDate")


for k in range(1,31):
    date = "2017.03."
    
    date = date + repr(k).zfill(2)
     
    url = url_part1+ date + url_part2+ repr(j)
    res = requests.get(url)
    _soup = bs(res.text,"html.parser")
    
    if _soup.find("p",class_="page_info")!=None:
        totNum=_soup.find("p",class_="page_info").text
        totNum=re.sub(r"\[.*\] \[총","",totNum)
        totNum=re.sub(r"건\]","",totNum)
        totNum=totNum.replace(",","")
        totNum = int(totNum)
        if totNum%100==0:
            totPage = (totNum//100)
        else:    
            totPage = (totNum//100)+1
    else:
        continue
   
    
    for j in range(1,totPage+1):
        
         
        url = url_part1 + date + url_part2 +repr(k)
        res = requests.get(url) 
        soup=bs(res.text, 'html.parser')

        index = soup.find_all("tr")

        for i in range(1,len(index)):

            df_temp.iat[0,cat_idx] = index[i].find_all("td")[1].find("img").get("title")
            df_temp.iat[0,conm_idx] = index[i].find_all("td")[1].text.strip()

            tempStr = index[i].find_all("td")[2].text.strip()
            tempStr = tempStr.replace("\r","")
            tempStr = tempStr.replace("\n","")
            tempStr = tempStr.replace("\t","")
            tempStr = tempStr.replace(" ","")

            df_temp.iat[0,con_idx] = tempStr


            testStr = index[i].find("a").get("href")
            re.sub(r"\/dsae001\/selectPopup.ax\?selectKey=", "", testStr)

            df_temp.iat[0,id_idx] = re.findall(r"[0-9]{8}", testStr)[0]
            df_temp.iat[0,pdate_idx] = date
            df_temp.iat[0,ptime_idx] = index[i].find_all("td")[0].text.strip()

            tempStr2 = index[i].find_all("td")[2].find("a").get("href")
            re.sub(r"\/dsaf001/main.do?rcpNo=","",tempStr2)

            df_temp.iat[0,rcp_idx] = re.findall(r"[0-9]{14}",tempStr2)[0]
            df_temp.iat[0,rdate_idx] = index[i].find_all("td")[4].text

            df = df.append(df_temp,ignore_index=True)
            df_temp.iloc[0,:]=""

df

NameError: name 'j' is not defined